In [6]:
import os, json, uuid
from datetime import datetime, timezone, timedelta
from pymongo import MongoClient, ReturnDocument
from pymongo.errors import DuplicateKeyError
from dotenv import load_dotenv
from bson.decimal128 import Decimal128

# ---------------------------
# Setup
# ---------------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"
if not uri or not database_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE.")
col = MongoClient(uri)[database_name][collection_name]

# ---------------------------
# Input (replace at runtime)
# ---------------------------
payload = {
    "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
    "current_form_qr_code": "20251013",
    "previous_form_qr_code": "20251012",
    "tin": "123-456-789",
    "location": "JEF Gas Station – Sikatuna Branch",
    "created": "2025-10-13T05:03:46.557977Z",
    "date": "2025-10-13",
    "items": [
        {"receipt_number": "123","customer_name": "Markus","type": "fuel","vatable_sales": 1111,"vat_amount": 133.32,"total_amount": 1244.32},
        {"receipt_number": "222","customer_name": "Just","type": "fuel","vatable_sales": 22222,"vat_amount": 2666.64,"total_amount": 24888.64}
    ],
    "cashier_employee_number": "10001",
    "cashier": "John Smith",
    "recorder_employee_number": "10003",
    "recorder": "David Williams"
}

# ---------------------------
# Small helpers
# ---------------------------
REQUIRED = [
    "id","current_form_qr_code","previous_form_qr_code","tin","location","date","items",
    "cashier_employee_number","cashier","recorder_employee_number","recorder"
]

def dec128(x): 
    return x if isinstance(x, Decimal128) else Decimal128(str(x))

def normalize_payload(p):
    p = dict(p)
    its = []
    for it in p.get("items", []):
        it = dict(it)
        for k in ("vatable_sales","vat_amount","total_amount"):
            if k in it and it[k] is not None:
                it[k] = dec128(it[k])
        its.append(it)
    p["items"] = its
    return p

def parse_date(d):  # accepts 'YYYY-MM-DD' or ISO datetime
    try: return datetime.fromisoformat(d).date()
    except Exception:
        try: return datetime.fromisoformat(d.replace("Z","+00:00")).date()
        except Exception: return datetime.now(timezone.utc).date()

# ---------------------------
# 1) Index + sync flags (idempotent)
# ---------------------------
try:
    col.create_index(
        [("is_empty",1)],
        unique=True,
        name="uniq_is_empty_true",
        partialFilterExpression={"is_empty": True},
        background=True,
    )
except Exception:
    pass

col.update_many({"items": {"$exists": False}}, {"$set": {"items": []}})
col.update_many({"items": {"$exists": True, "$size": 0}}, {"$set": {"is_empty": True}})
col.update_many({"items": {"$exists": True, "$ne": []}}, {"$set": {"is_empty": False}})

# ---------------------------
# 2) Validate + normalize payload
# ---------------------------
missing = [k for k in REQUIRED if k not in payload]
if missing: raise RuntimeError(f"❌ Missing fields: {missing}")
if not isinstance(payload.get("items"), list) or not payload["items"]:
    raise RuntimeError("❌ Payload.items must be a non-empty list.")
norm = normalize_payload(payload)

# ---------------------------
# 3) Patch the only empty doc
# ---------------------------
if col.count_documents({"is_empty": True}) != 1:
    cnt = col.count_documents({"is_empty": True})
    raise RuntimeError(f"❌ Expected exactly one is_empty=True doc; found {cnt}.")

patched = col.find_one_and_update(
    {"is_empty": True},
    {"$set": {
        "id": norm["id"],
        "current_form_qr_code": norm["current_form_qr_code"],
        "previous_form_qr_code": norm["previous_form_qr_code"],
        "tin": norm["tin"],
        "location": norm["location"],
        "date": norm["date"],
        "cashier_employee_number": norm["cashier_employee_number"],
        "cashier": norm["cashier"],
        "recorder_employee_number": norm["recorder_employee_number"],
        "recorder": norm["recorder"],
        "items": norm["items"],
        "updated_at": datetime.now(timezone.utc).isoformat(),
        "is_empty": False
    }},
    return_document=ReturnDocument.AFTER
)
if not patched:
    raise RuntimeError("❌ Patch lost race; try again.")

print("✅ Patched:")
print(json.dumps(patched, indent=2, default=str))

# ---------------------------
# 4) Create the next empty doc
# ---------------------------
prev_date = parse_date(str(patched.get("date","")))
next_date = (prev_date + timedelta(days=1)).isoformat()
now = datetime.now(timezone.utc)

new_empty = {
    "id": str(uuid.uuid4()),
    "previous_form_qr_code": patched.get("current_form_qr_code",""),
    "current_form_qr_code":"12345678",
    "tin": patched.get("tin",""),
    "location": patched.get("location",""),
    "created": now.isoformat(),
    "date": next_date,
    "is_empty": True,
}

try:
    ins = col.insert_one(new_empty)
    new_empty["_id"] = str(ins.inserted_id)
except DuplicateKeyError:
    new_empty = col.find_one({"is_empty": True}, sort=[("created",-1)])

print("✅ Fresh empty (ready for next run):")
print(json.dumps(new_empty, indent=2, default=str))


✅ Patched:
{
  "_id": "68ed4cc1d4cf7cfbc7b74138",
  "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "20251013",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T19:02:25.913240+00:00",
  "date": "2025-10-13",
  "is_empty": false,
  "items": [
    {
      "receipt_number": "123",
      "customer_name": "Markus",
      "type": "fuel",
      "vatable_sales": "1111",
      "vat_amount": "133.32",
      "total_amount": "1244.32"
    },
    {
      "receipt_number": "222",
      "customer_name": "Just",
      "type": "fuel",
      "vatable_sales": "22222",
      "vat_amount": "2666.64",
      "total_amount": "24888.64"
    }
  ],
  "cashier": "John Smith",
  "cashier_employee_number": "10001",
  "recorder": "David Williams",
  "recorder_employee_number": "10003",
  "updated_at": "2025-10-13T19:02:37.051421+00:00"
}
✅ Fresh empty (ready for next run):
{
  "id": "fc4eb602-6f37